In [35]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer,StopWordsRemover,CountVectorizer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import TrainValidationSplit,ParamGridBuilder
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType

def removePunctuation(text):
    text = text.replace('.','')
    text = text.replace(',','')
    text = text.replace(';','')
    text = text.replace(':','')
    text = text.replace('ç','c')
    text = text.replace('ş','s')
    return text

spark = SparkSession.builder.getOrCreate()
textDF = spark.read.option('delimiter','\t').option('inferSchema','true').csv('datasets/movie_turkish_train.txt')
textDF = textDF.withColumnRenamed('_c0','orig')
textDF = textDF.withColumnRenamed('_c1','label')

myUDF = UserDefinedFunction(removePunctuation,StringType())

textDF = textDF.withColumn('Text',myUDF('orig'))


textDF.show()
tokenizer = Tokenizer(inputCol='Text',outputCol='tokenized')
textDF = tokenizer.transform(textDF)

trStopWords = StopWordsRemover.loadDefaultStopWords('turkish')
sRemover = StopWordsRemover(inputCol='tokenized',outputCol='removed',stopWords=trStopWords)
textDF = sRemover.transform(textDF)

vectorizer = CountVectorizer(inputCol='removed',outputCol='features',vocabSize=3000)
textDF = vectorizer.fit(textDF).transform(textDF)

textDF = textDF.select('features','label')
trainDF, testDF = textDF.randomSplit([0.75,0.25],seed=123) 

rfClassifier= RandomForestClassifier()
model = rfClassifier.fit(trainDF)

resultDF = model.transform(testDF)

eva = BinaryClassificationEvaluator()
successRate = eva.evaluate(resultDF)
print("Accuracy : ",successRate)
resultDF.show()

+--------------------+-----+--------------------+
|                orig|label|                Text|
+--------------------+-----+--------------------+
|bunu da gördük uz...|    0|bunu da gördük uz...|
|filmi begenmeyenl...|    0|filmi begenmeyenl...|
|günde 1 film izle...|    0|günde 1 film izle...|
|afisine bakip ald...|    0|afisine bakip ald...|
|sadece insanlarin...|    0|sadece insanlarin...|
|ucuz bir aksiyon ...|    0|ucuz bir aksiyon ...|
|olmamis diyor pua...|    0|olmamis diyor pua...|
|kesinlikle çok kö...|    0|kesinlikle cok kö...|
|ben nasil bir fil...|    0|ben nasil bir fil...|
|bu yüzler fazla t...|    0|bu yüzler fazla t...|
|. çok ilginç yaa ...|    0| cok ilginc yaa k...|
|bence bu film hiç...|    0|bence bu film hic...|
|valla ben begenme...|    0|valla ben begenme...|
|gönül ister milyo...|    0|gönül ister milyo...|
|dün aksam bu film...|    0|dün aksam bu film...|
|cok kötü olmus......|    0|cok kötü olmus am...|
|igrenç yaaa.acaba...|    0|igrenc yaaaacaba ...|


In [ ]:
import pyspark.sql.functions as F

df2 = df.select(
    [F.regexp_replace(col, r',|\.|&|\\|\||-|_', '').alias(col) for col in df.columns]
)